## Modifying Django admin interface

If we use **custom user model, the admin page must also be adapted for the changes**. The following changes have to be applied

> 1) **Fields to be displayed** in the **users list** page

> 2) **Fields to be displayed** in the **edit user** page

> 3) **Fields needed** to **add user**

### Test cases to check functionalities

Find the urls of those 3 pages from django docs, and check if these page load properly (response 200)

> 1) Users list page 
```python
url = reverse('admin:core_user_changelist')
```

>2) Users edit page
```python
url = reverse('admin:core_user_change', args=[self.user.id])
# /admin/core/user/1
```

> 3) Users add page
```python
url = reverse('admin:core_user_add')
```

<div style="border:2px solid;margin:0 20px 0 20px" ></div> 

```python
from django.test import TestCase, Client
from django.contrib.auth import get_user_model
from django.urls import reverse

class AdminSiteTest(TestCase):

    # Setup function is run before every test
    # create a super user, log him in and create a normal user
    def setUp(self):
        """Create a super user and log him in
            Create a user"""
        self.client = Client()
        self.admin_user = get_user_model().objects.create_superuser(
            email='aravind@gmail.com',
            password='1234'
        )
        self.client.force_login(self.admin_user)
        self.user = get_user_model().objects.create_user(
            email='silent@retreat.com',
            password='1234',
            name='Test user name'
        )

    def test_users_listed(self):
        """Test that users are listed in user page.
        We have to make changes to django admin to accomodate
        ou custom user model"""

        # These urls are listed in django admin docs
        # Gets the URL that lists users in admin page
        # have to register User model to admin for this url to work
        url = reverse('admin:core_user_changelist')
        response = self.client.get(url)

        # AssertContains checks if certain value is present in a dict
        # Also checks if the http respose is OK (200)
        # name field is not available in default UserAdmin class
        self.assertContains(response, self.user.name)

    def test_user_page_change(self):
        """ Test that user edit page works"""

        # We have to include fieldssets to UserAdmin for this to work
        url = reverse('admin:core_user_change', args=[self.user.id])
        # /admin/core/user/1
        response = self.client.get(url)

        self.assertEqual(response.status_code, 200)

    def test_create_user_page(self):
        url = reverse('admin:core_user_add')
        response = self.client.get(url)

        self.assertEqual(response.status_code, 200)
```
<div style="border:2px solid;margin:0 20px 0 20px" ></div> 


### Implementation.

In **admin.py**, Create a class **UserAdmin** extending the default UserAdmin class**. Define the fields for the corresponding pages

<div style="border:2px solid;margin:0 20px 0 20px" ></div> 

```python
from django.contrib import admin
from django.contrib.auth.admin import UserAdmin as BaseUserAdmin
from core import models
# this hook is needed to make django projects translatable,
# Wrap the texts with this if you want django to automatically translate
from django.utils.translation import gettext as _


# This class is used just for the admin interface.
# Nothing changes except the way admin page looks
class UserAdmin(BaseUserAdmin):
    ordering = ['id']

    # fields to be included in list users page
    list_display = ['email', 'name']

    # fields to be included on change user page (edit page)
    fieldsets = (
        (None, {'fields': ('email', 'password')}),
        (_('Personal Info'), {'fields': ('name',)}),
        (_('Permissions'),
            {'fields': ('is_active', 'is_staff', 'is_superuser')}
         ),
        (_('Important dates'), {'fields': ('last_login',)})
    )

    # fields to be included in add user page
    # therefore we can create a new user with email and password
    add_fieldsets = (
        (None, {
            'classes': ('wide',),
            'fields': ('email', 'password1', 'password2')
        }),
    )


# If we use the default admin class, we dont have to pass the second parameter
# Here we modified the default adin class
admin.site.register(models.User, UserAdmin)
```
<div style="border:2px solid;margin:0 20px 0 20px" ></div> 


### Page display

#### List user page

<img src="files/listuser.png" />

#### Edit user page

<img src="files/changeuser.png" />

#### Add user page

<img src="files/adduser.png" />